In [1]:
import pandas as pd
import numpy as np

PATH = '/home/kai/data/kaggle/homecredit/'
train = pd.read_csv(PATH + 'application_train.csv')
print('train')
test = pd.read_csv(PATH + 'application_test.csv')
print('test')
# bureau = pd.read_csv(PATH + 'bureau.csv')
# print('bureau')
# previous_application = pd.read_csv(PATH + 'previous_application.csv')
# print('previous')
# bureau_balance = pd.read_csv(PATH + 'bureau_balance.csv')
# print('bureau_balance')
# installments_payments = pd.read_csv(PATH + 'installments_payments.csv')
# print('installments_payments done')
# credit_card_balance = pd.read_csv(PATH + 'credit_card_balance.csv')
# # print('credit_card_balance')
POS_CASH_balance = pd.read_csv(PATH + 'POS_CASH_balance.csv')
print('POS_CASH_balance')
# credit_card_balance = pd.read_csv(PATH + 'credit_card_balance.csv')
# print('credit_card_balance')
# previous_application = pd.read_csv(PATH + 'previous_application.csv')
# print('previous_application')

train
test
POS_CASH_balance


In [2]:
df = pd.concat([train,test], axis = 0)
df = df[['SK_ID_CURR']]
pos = POS_CASH_balance.copy()

In [3]:
import pandas as pd
import os
import gc
import numpy as np

def ratio_name(numerator, denominator): return numerator + '_divide_' + denominator

def ratio(df, numerator, denominator):
    return df[numerator] / df[denominator]

def substraction_name(col1, col2): return col1 + '_minus_' + col2

def substraction(df, col1, col2):
    return df[col1] - df[col2]


# Substraction and ratio , and score for dpd

In [4]:
pos[substraction_name('CNT_INSTALMENT','CNT_INSTALMENT_FUTURE')] = \
substraction(pos, 'CNT_INSTALMENT','CNT_INSTALMENT_FUTURE')

pos[ratio_name('CNT_INSTALMENT_FUTURE','CNT_INSTALMENT')]\
= ratio(pos,'CNT_INSTALMENT_FUTURE','CNT_INSTALMENT')

In [5]:
pos['score_DPD'] = 0
pos = pos.sort_values('MONTHS_BALANCE')
pos['score_DPD'] = (pos['SK_DPD']**3 + 5* pos['SK_DPD_DEF']**3 + 1) * (1/(1 + abs(pos['MONTHS_BALANCE']) ))

In [6]:
pos.columns.values

array(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'CNT_INSTALMENT',
       'CNT_INSTALMENT_FUTURE', 'NAME_CONTRACT_STATUS', 'SK_DPD',
       'SK_DPD_DEF', 'CNT_INSTALMENT_minus_CNT_INSTALMENT_FUTURE',
       'CNT_INSTALMENT_FUTURE_divide_CNT_INSTALMENT', 'score_DPD'],
      dtype=object)

# Numerical

In [7]:
def numerical_agg(df, gp_col, agg_col, df_name, extrafunc_list = None):
    
    agg_list = ['max', 'min', 'std','mean']
    '''
    Possible extrafunc_list: ['sum','median','two_minus_one_third','positive_count', 'negative_count','standard_error', 'trimmed_mean_10_pct', 'trimmed_mean_25_pct','normed_std', 'max_minus_min','one_third','two_third']
    '''
    if extrafunc_list:
        if 'one_third' in extrafunc_list:
            def one_third(series):
                return series.quantile(1/3)
            
        if 'two_third' in extrafunc_list:
            def two_third(series):
                return series.quantile(2/3)
            
        if 'max_minus_min' in extrafunc_list:
            def max_minus_min(series):
                return (series.max() - series.min())
        
        if 'two_minus_one_third' in extrafunc_list:
#             print('two_minus_one')
#             print('get two minus one')
            def two_minus_one_third(series):
                return(series.quantile(0.66666) - series.quantile(0.33333))
       
        if 'positive_count' in extrafunc_list:
#             print('get positive count')
            def positive_count(series):
                return pd.Series(series > 0).sum()

        if 'negative_count' in extrafunc_list:
#             print('get negative count')
            def negative_count(series):
                return pd.Series(series < 0).sum()

        if 'standard_error' in extrafunc_list:
#             print('get std-error')
            def standard_error(series):
                return series.std()/np.sqrt(len(series))

        if 'normed_std' in extrafunc_list:
#             print('get normed_std')
            def normed_std(series):
                return series.std()/series.mean()

        if 'trimmed_mean_10_pct' in extrafunc_list:
#             print('get_trimmed 10 pct')
            def trimmed_mean_10_pct(series):
                return stats.trim_mean(series.dropna(), 0.1)

        if 'trimmed_mean_25_pct' in extrafunc_list:
#             print('get_trimmed 25 pct')
            def trimmed_mean_25_pct(series):
                return stats.trim_mean(series.dropna(), 0.25)
        
        list_tocall = []
        for i in extrafunc_list:
            if i not in set(['sum', 'median']):
                list_tocall.append(eval(i))
            
    if extrafunc_list != None:   
        agg_list.extend(list_tocall)
    
    _df = df.groupby(gp_col).agg({agg_col:agg_list})
    columns = []
    for pre in _df.columns.levels[0]:
        for middle in _df.columns.levels[1]:
            columns.append('%s_%s_%s' %(df_name,pre,middle))
    _df.columns = columns
    
    return _df.reset_index()

In [8]:
gp_col = 'SK_ID_CURR'
numerical_cols= ['CNT_INSTALMENT','CNT_INSTALMENT_FUTURE','SK_DPD','SK_DPD_DEF',\
                 'CNT_INSTALMENT_minus_CNT_INSTALMENT_FUTURE','CNT_INSTALMENT_FUTURE_divide_CNT_INSTALMENT','score_DPD']

'''
    Possible extrafunc_list: ['sum','median','two_minus_one_third','positive_count', 'negative_count','standard_error', 'trimmed_mean_10_pct', 'trimmed_mean_25_pct','normed_std', 'max_minus_min','one_third','two_third']
    '''
extrafunc_list = [None,None,['positive_count'],['positive_count'],None,None, None]

for i in range(len(numerical_cols)):
    print(numerical_cols[i])
    df = df.merge(numerical_agg(pos, gp_col, numerical_cols[i], 'POS_CASH_balance', extrafunc_list = extrafunc_list[i]), on = 'SK_ID_CURR', how='left')

CNT_INSTALMENT
CNT_INSTALMENT_FUTURE
SK_DPD
SK_DPD_DEF
CNT_INSTALMENT_minus_CNT_INSTALMENT_FUTURE
CNT_INSTALMENT_FUTURE_divide_CNT_INSTALMENT
score_DPD


# Categorical Features

In [9]:
def split_categorical_feature(df, group_col, calc_col):
    tmp_df = pd.concat([df[group_col], pd.get_dummies(df[calc_col], prefix=calc_col)], axis=1).groupby(by=group_col).sum().reset_index()
    dtype = {x: _set_type(tmp_df[x], 'uint') for x in tmp_df.columns if x != group_col}
    for x in tmp_df:
        if x in df.columns:
            dtype[x] = df[x].dtype
    return tmp_df.astype(dtype)

def _set_type(series, dtype):
    _max, _min = max(series), min(series)
    if dtype == 'uint':
        if _max <= 255: return np.uint8
        elif _max <= 65535: return np.uint16
        elif _max <= 4294967295: return np.uint32
        else: return np.uint64
    elif dtype == 'int':
        if _min >= -128 and _max <= 127: return np.int8
        elif _min >=-32768 and _max <= 32767: return np.int16
        elif _min >= -2147483648 and _max <= 2147483647: return np.int32
        else: return np.int64
    elif dtype == 'float':
        if max(abs(_min), _max) <= 3.4028235e+38: return np.float32
        else: return np.float64

In [10]:
categorical_cols = ['NAME_CONTRACT_STATUS']        

for x in categorical_cols:
    print(x)
    df = df.merge(split_categorical_feature(pos, 'SK_ID_CURR', x), on='SK_ID_CURR', how='left')
    gc.collect()

NAME_CONTRACT_STATUS


In [13]:
_df = POS_CASH_balance.groupby('SK_ID_CURR').agg({'MONTHS_BALANCE':['sum','min','max']})
columns = []
for pre in _df.columns.levels[0]:
    for middle in _df.columns.levels[1]:
        columns.append('%s_%s_%s' %('POS_CASH_balance',pre,middle))
_df.columns = columns
df = df.merge(_df.reset_index(), on = 'SK_ID_CURR', how='left')

In [16]:
print(df.columns.values, len(df.columns.values))

['SK_ID_CURR' 'POS_CASH_balance_CNT_INSTALMENT_max'
 'POS_CASH_balance_CNT_INSTALMENT_min'
 'POS_CASH_balance_CNT_INSTALMENT_std'
 'POS_CASH_balance_CNT_INSTALMENT_mean'
 'POS_CASH_balance_CNT_INSTALMENT_FUTURE_max'
 'POS_CASH_balance_CNT_INSTALMENT_FUTURE_min'
 'POS_CASH_balance_CNT_INSTALMENT_FUTURE_std'
 'POS_CASH_balance_CNT_INSTALMENT_FUTURE_mean'
 'POS_CASH_balance_SK_DPD_max' 'POS_CASH_balance_SK_DPD_min'
 'POS_CASH_balance_SK_DPD_std' 'POS_CASH_balance_SK_DPD_mean'
 'POS_CASH_balance_SK_DPD_positive_count'
 'POS_CASH_balance_SK_DPD_DEF_max' 'POS_CASH_balance_SK_DPD_DEF_min'
 'POS_CASH_balance_SK_DPD_DEF_std' 'POS_CASH_balance_SK_DPD_DEF_mean'
 'POS_CASH_balance_SK_DPD_DEF_positive_count'
 'POS_CASH_balance_CNT_INSTALMENT_minus_CNT_INSTALMENT_FUTURE_max'
 'POS_CASH_balance_CNT_INSTALMENT_minus_CNT_INSTALMENT_FUTURE_min'
 'POS_CASH_balance_CNT_INSTALMENT_minus_CNT_INSTALMENT_FUTURE_std'
 'POS_CASH_balance_CNT_INSTALMENT_minus_CNT_INSTALMENT_FUTURE_mean'
 'POS_CASH_balance_CNT_INS